<a href="https://colab.research.google.com/github/panpapap/Python_portfolio/blob/main/SQL_Project_Chinook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Introduction
This project aims to analyze and answer key business questions about a fictional music record store called Chinook.
The database could be found [here](https://github.com/panpapap/Python_portfolio/tree/main/datasets/SQL).

**Skills used:** sub queries, INNER JOIN, GROUP BY, ORDER BY, WHERE, WITH

###Database Schema
The schema diagram for the Chinook database looks like the following:
<div>
<img src="https://i.imgur.com/dq79HLP.png" width="500" alt = "Chinook database schema"/>
</div>

In [22]:
#import sql3, pandas and connect to the databse.
import sqlite3
import pandas as pd
conn = sqlite3.connect("/content/drive/MyDrive/Portfolio_files/SQL/chinook.db")

#activates the cursor
cursor = conn.cursor()

#the SQL query to look at the tables in the databse
q = "SELECT * FROM sqlite_master WHERE type='table';"

#execute the query and read it in pandas, this returns a table in pandas form
db = pd.read_sql_query(q, conn)
db

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


#Business Question #1 - **Top selling genres**
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
<div align ="center">

|Artist Name|	Genre|
|--|--|
|Regal	|Hip-Hop|
|Red Tone|	Punk|
|Meteor and the Girls	|Pop|
|Slim Jim Bites	|Blues|

<div align ="left">

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in *finding out which genres sell the best in the USA.*

To solve this problem, I will have to SELECT genres, total quantity sold for USA record labels. 

In [33]:
q1 = '''SELECT
         g.name AS genre,
         SUM(il.quantity) AS total_sold,
         SUM(CAST(il.quantity AS FLOAT)) /
            (SELECT SUM(il.quantity) AS total_tracks_sold 
             FROM invoice_line AS il
             INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
             WHERE i.billing_country = "USA") AS percent_sold
     FROM track AS t
     INNER JOIN genre AS g ON g.genre_id = t.genre_id
     INNER JOIN invoice_line AS il ON il.track_id = t.track_id
     INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
     WHERE i.billing_country = "USA"
     GROUP BY genre
     ORDER BY 2 DESC
     LIMIT 10;
     '''
dt = pd.read_sql_query(q1, conn)
dt

,genre,total_sold,percent_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


In the U.S.A, the sales for Rock genre is significant. It accounts for approximatly 55% of the total track sales. Follow by that are Metal and Alternative & Punk with 12.4% and 11.8% respectively.

To answer our business question, of which new artists should we signed with, I would recommend Red Tone (Punk), Slim Jim Bites (Blues).
On the other hand, Meteor and the Girls (Pop) and Regal (Hip-Hop) should not be chosen as these two artists' genres were not selling well in the States. 

#Business Question #2 - Sales per Agent
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to *analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.*


You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [38]:
q2 = '''SELECT
               e.first_name || ' ' || e.last_name AS employee,
               COUNT( DISTINCT c.customer_id) AS num_customer,
               SUM(i.total) AS sales_amount,
               SUM(i.total) / COUNT( DISTINCT c.customer_id) AS avg_sales_per_customer,
               SUM(i.total)/ (SELECT SUM(i.total) FROM invoice AS i) AS percent_sales_amount
           FROM employee AS e
           LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id 
           LEFT JOIN invoice as i ON i.customer_id = c.customer_id
           GROUP BY e.employee_id
           ORDER BY 3 DESC
           LIMIT 5;
           '''

dt = pd.read_sql_query(q2, conn)
dt

,employee,num_customer,sales_amount,avg_sales_per_customer,percent_sales_amount
0,Jane Peacock,21,1731.51,82.452857,0.367669
1,Margaret Park,20,1584.00,79.200000,0.336346
2,Steve Johnson,18,1393.92,77.440000,0.295985
3,Andrew Adams,0,NaN,NaN,NaN
4,Nancy Edwards,0,NaN,NaN,NaN


From the table above, we could see that all the sales focus on 3 agents: Jane Peacock, Margaret Park and Steve Johnson. 

Jane Peacock has made the highest amount of sales so far, with a 36.8% of the total, followed by Margaret Park and Steve Johnson, with a 33.6% and 29.6% respectively. 

The range of average sales per customer, however, is not far apart (77 to 82) thus there are no noteable differences in three employees' performances.

#Business Question #3 - Sales by Countries
Your next task is to analyze the *sales data for customers from each different country*. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [39]:
q31 = ''' SELECT
               c.country AS country_by_cust_table,
               COUNT( DISTINCT c.customer_id) AS num_customers,
               SUM(i.total) AS total_sales,
               ROUND(SUM(i.total) / COUNT( DISTINCT c.customer_id),2) AS avg_sale_per_cust,
               COUNT(i.total) AS num_orders,
               ROUND(AVG(i.total), 2) AS avg_order_value
           FROM customer AS c
           INNER JOIN invoice AS i ON i.customer_id = c.customer_id
           GROUP BY c.country 
           ORDER BY total_sales DESC
            '''

dt = pd.read_sql_query(q31, conn)
dt

,country_by_cust_table,num_customers,total_sales,avg_sale_per_cust,num_orders,avg_order_value
0,USA,13,1040.49,80.04,131,7.94
1,Canada,8,535.59,66.95,76,7.05
2,Brazil,5,427.68,85.54,61,7.01
3,France,5,389.07,77.81,50,7.78
4,Germany,4,334.62,83.66,41,8.16
5,Czech Republic,2,273.24,136.62,30,9.11
6,United Kingdom,3,245.52,81.84,28,8.77
7,Portugal,2,185.13,92.57,29,6.38
8,India,2,183.15,91.57,21,8.72
9,Ireland,1,114.84,114.84,13,8.83


Of the 24 countires that have customers in them, 15 of them have only one customer in them. These countries will be grouped together into an Other category.



In [40]:
q32 = ''' WITH country_and_other AS
           (
           SELECT
           CASE
               WHEN (
                     SELECT count(*)
                     FROM customer
                     where country = c.country
                    ) = 1 THEN "Other"
               ELSE c.country
           END AS country,
           c.customer_id,
           i.*
           FROM invoice AS i
           INNER JOIN customer c ON c.customer_id = i.customer_id
           ) 
           
           SELECT
               country,
               num_customers,
               total_sales,
               avg_sale_per_cust,
               num_orders,
               avg_order_value
           FROM
               (
                SELECT
                    c.country AS country,
                    COUNT( DISTINCT c.customer_id) AS num_customers,
                    SUM(c.total) AS total_sales,
                    ROUND(SUM(c.total) / COUNT( DISTINCT c.customer_id),2) AS avg_sale_per_cust,
                    COUNT(c.total) AS num_orders,
                    ROUND(AVG(c.total), 2) AS avg_order_value,
                    CASE
                        WHEN country = "Other" THEN 1 
                        ELSE 0 
                    END AS sort
                FROM country_and_other AS c 
                GROUP BY c.country
                ORDER BY sort, total_sales DESC)
            '''

dt = pd.read_sql_query(q32, conn)
dt

,country,num_customers,total_sales,avg_sale_per_cust,num_orders,avg_order_value
0,USA,13,1040.49,80.04,131,7.94
1,Canada,8,535.59,66.95,76,7.05
2,Brazil,5,427.68,85.54,61,7.01
3,France,5,389.07,77.81,50,7.78
4,Germany,4,334.62,83.66,41,8.16
5,Czech Republic,2,273.24,136.62,30,9.11
6,United Kingdom,3,245.52,81.84,28,8.77
7,Portugal,2,185.13,92.57,29,6.38
8,India,2,183.15,91.57,21,8.72
9,Other,15,1094.94,73.00,147,7.45


According to the table, the United States and Canada are the company's largest markets in terms of overall sales, followed by countries such as Brazil and France.

However, based on the number of customers, the amount of money spent by unique users thus far, and the average order values, **Czech Republic, the UK, India and Portugal** have a large growth potential, since they have less customers that, on average, spend a higher total and per order amount of money.

#Business Question #4 - Purchasing Strategy: Album vs. Individual Tracks
Management is currently considering changing their *purchasing strategy* to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. We have been asked to find out *what percentage of purchases are individual tracks vs whole albums*, so that management can use this data to understand the effect this decision might have on overall revenue.

In [41]:
q4 = '''
    WITH    
        album_invoice AS
                (
                SELECT 
                    il.invoice_id,
                    il.track_id,
                    t.album_id
                FROM invoice_line il
                INNER JOIN track t ON t.track_id = il.track_id
                ),
        
        
        invoice_info AS
                (
                SELECT 
                    invoice_id,
                    COUNT(DISTINCT album_id) num_albums,
                    COUNT(track_id) num_tracks,
                    CASE
                        COUNT(DISTINCT album_id)
                        WHEN 1 THEN album_id
                        ELSE NULL
                        END AS album_id
                FROM album_invoice
                GROUP BY invoice_id
                ),
            
            
        track_album AS
                (
                SELECT 
                    COUNT(track_id) num_tracks,
                    album_id
                FROM track t
                WHERE album_id IN (
                                  SELECT album_id FROM invoice_info
                                  WHERE num_albums = 1    
                                 )
                GROUP BY album_id
                ORDER BY album_id ASC
                )
          
       
       
    SELECT 
        album_purchase,
        COUNT(invoice_id) num_invoices,
        CAST(COUNT(invoice_id) as float)/(SELECT COUNT(*) FROM invoice) percent 
    FROM
      (
        SELECT
            invoice_id,
            CASE
                WHEN (ii.album_id == ta.album_id AND ii.num_tracks == ta.num_tracks) THEN 'Yes'
                ELSE 'No'
                END AS album_purchase
        FROM invoice_info ii
        LEFT JOIN track_album ta ON ii.album_id = ta.album_id
      ) 
    GROUP BY album_purchase;

'''

dt = pd.read_sql_query(q4, conn)
dt

,album_purchase,num_invoices,percent
0,No,500,0.814332
1,Yes,114,0.185668


Based on the above table 18.6% of customer purchases are full album purchases, while the majorities 81.4% are buying individual tracks.